In [50]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from scipy.stats import norm

# Value at Risk (VaR)

## Introduction

VaR was developed in mid-1990s, in response to the various financial crisis, but the origins of the measures lie further back in time. According to Philippe Jorion, “VaR measures the worst expected loss over a given horizon under normal market conditions at a given level of confidence”.

This definition implies that it is necessary to choose two parameters, namely holding period and confidence level. Typically the confidence level lies in the range 90% to 99% such as 90%, 95%, 99%. Holding period may vary from a day to a year.

Suppose, an analyst says that the 1-day VaR of a portfolio is £1,000,000 with a 95% confidence level. It implies there is 95% chance that the maximum losses will not exceed £1,000,000 in a single day. In other words, there is only 5% chance that the portfolio losses on a particular day will be greater than £1,000,000.

The Variance-Covariance is a parametric method which assumes that the returns are normally distributed.

[https://www.quantopian.com/posts/quantopian-lecture-series-var-and-cvar-expected-shortfall](https://www.quantopian.com/posts/quantopian-lecture-series-var-and-cvar-expected-shortfall)

In [2]:
# Get historical close data for JP Morgan
df = pdr.get_data_yahoo('JPM')

### 1) Calculate the Returns

In [73]:
df['% Returns'] = df['Adj Close'].pct_change()

### 2) Calculate the Mean

In [75]:
mu = np.mean(df['% Returns'])

### 3) Calculate the Standard Deviation

In [74]:
sigma = np.std(df['% Returns'])

### 4) Calculate VaR

Consider a portfolio of $P$ pounds, with a confidence level $c$. We are considering daily returns, with asset (or strategy) historical standard deviation $\sigma$ and mean $\mu$. Then the daily VaR, under the variance-covariance method for a single asset (or strategy) is calculated as:

$\alpha = \Phi^{-1}(1 − c)$

$V = P − P(\alpha + 1)$

__Where:__ 

$\alpha$ is the alpha

$\Phi$ is the inverse of the cumulative distribution function of a normal distribution

$\mu$ is the mean

$\sigma$ is the standard deviation

In [87]:
@widgets.interact(portfolio_value=(1000000., 10000000., 100000.), confidence_interval=(0.01, 0.99, 0.01), 
                  mu=(0.0001, 0.001, 0.0001), sigma=(0.01, 0.02, 0.001))
def value_at_risk(portfolio_value, confidence_interval, mu, sigma):
    """Calculates VaR using the Variance-Covariance Model."""
    alpha = norm.ppf(1 - confidence_interval, mu, sigma)
    value_at_risk = portfolio_value - (portfolio_value * (alpha + 1))
    
    print(f'Alpha: {alpha}')
    print(f'VaR: £{value_at_risk}')

interactive(children=(FloatSlider(value=5500000.0, description='portfolio_value', max=10000000.0, min=1000000.…

### 5) Visualise the Distribution

In [92]:
@widgets.interact(confidence_level=(0.01, 0.99, 0.02))
def visualise(confidence_level):
    df['% Returns'].hist(bins=40, density=True, histtype='stepfilled', alpha=0.5)
    x = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
    value_at_risk = norm.ppf(1 - confidence_level, mu, sigma)
    plt.axvline(value_at_risk, color='red', linestyle='solid')
    plt.plot(x, norm.pdf(x, mu, sigma), 'g')

interactive(children=(FloatSlider(value=0.49, description='confidence_level', max=0.99, min=0.01, step=0.02), …

### 6) Jarque Bera Test

In [72]:
from statsmodels.stats.stattools import jarque_bera

_, pvalue, _, _ = jarque_bera(df['% Returns'])

if pvalue > 0.05:
    print('The portfolio returns are likely normal.')
else:
    print('The portfolio returns are likely not normal.')

The portfolio returns are likely not normal.
